# Code Similarity, AI Detection, and Plagiarism Analysis Notebook

This notebook provides a comprehensive pipeline for analyzing Python code submissions using advanced machine learning and NLP techniques. It is designed for:

- **AI-generated code detection**: Uses GPT-2 language model to estimate code perplexity and pattern analysis to identify AI-generated code.
- **Plagiarism detection**: Compares code against common algorithmic patterns and known sources using fast hashing and similarity matching.
- **Batch and parallel analysis**: Supports efficient batch processing and parallel execution for large-scale code review.
- **Performance metrics**: Includes benchmarking for single and batch analysis, with cache optimization for repeated code checks.
- **Device selection**: Automatically detects and utilizes GPU (CUDA) if available for faster inference, otherwise falls back to CPU.
- **Model saving**: Demonstrates saving HuggingFace models and tokenizers for production deployment.

## Key Components
- **OptimizedAIGeneratedCodeDetector**: Singleton class for AI detection using GPT-2, with caching and fast pattern analysis.
- **OptimizedPlagiarismDetector**: Detects plagiarism by matching code against pre-computed hashes and known patterns.
- **OptimizedCodeChecker**: Integrates AI and plagiarism detection, supports parallel and batch analysis, and provides actionable recommendations.
- **Test Suite**: Performance tests for single and batch code analysis, including suspicious, plagiarized, and original code examples.

## Usage
1. **Check Python environment and device**: Ensure required libraries are installed and GPU is available for optimal performance.
2. **Import libraries and classes**: All dependencies are imported and classes are defined for immediate use.
3. **Run analysis**: Use the provided test suite or custom code snippets to analyze for AI generation and plagiarism.
4. **Save models**: Save trained or pre-trained models for future use or deployment.

## Requirements
- Python 3.8+
- PyTorch
- Transformers (HuggingFace)
- NumPy, requests, pymongo

---

> **Note:** This notebook is optimized for speed, scalability, and production-readiness. All detection logic is modular and can be integrated into backend services or automated code review pipelines.

In [1]:
import sys
print(sys.executable)

e:\project\ML\syntax_env\python.exe


### Device Selection and GPU Availability

This notebook is optimized to run on systems with a CUDA-enabled GPU for faster code analysis and AI detection. The device check below will automatically detect and display the available GPU. If no GPU is found, the notebook will default to CPU execution.

- **Why GPU?** GPU acceleration significantly speeds up model inference and batch processing, making large-scale code review much more efficient.
- **Automatic Detection:** The code cell checks for GPU availability and prints the device name if found, or notifies you if only CPU is available.

> **Recommendation:** For best performance, run this notebook on a machine with a CUDA-enabled GPU.

In [7]:
import torch
gpu_available = torch.cuda.is_available()
if gpu_available:
    print(f"GPU available: {torch.cuda.get_device_name(torch.cuda.current_device())}")
else:
    print("No GPU available")

GPU available: NVIDIA GeForce GTX 1650


### Project Dependencies

This notebook uses a combination of **data processing**, **AI/ML**, **NLP**, and **database** libraries for code analysis and detection tasks.

**Core Libraries:**
- `numpy` – Numerical computations and array operations  
- `torch` – PyTorch for deep learning and model inference  
- `transformers` – Hugging Face models and tokenizers (`AutoTokenizer`, `AutoModel`, `GPT2LMHeadModel`, `GPT2Tokenizer`)  

**Code Analysis & Processing:**
- `re` – Regular expressions for pattern matching  
- `ast` – Abstract Syntax Tree parsing for code inspection  
- `hashlib` – Generating hashes for code fingerprinting  
- `difflib` – Sequence matching for similarity detection  

**Performance & Utilities:**
- `functools.lru_cache` – Caching for faster repeated computations  
- `concurrent.futures.ThreadPoolExecutor` – Parallel execution  
- `warnings` – Suppressing unnecessary warnings  

**Web & Database:**
- `requests` – Sending HTTP requests  
- `pymongo.MongoClient` – Interacting with MongoDB  


> ⚡ **Tip:** Importing these libraries upfront ensures smooth execution for code analysis, AI detection, and database operations.


In [1]:
import numpy as np
import torch
import re
import ast
import hashlib
import difflib
from transformers import AutoTokenizer, AutoModel, GPT2LMHeadModel, GPT2Tokenizer
from functools import lru_cache
from concurrent.futures import ThreadPoolExecutor
import warnings
warnings.filterwarnings('ignore')

e:\project\ML\syntax_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


---

## AI-Generated Code Detection Logic

This section explains the logic and methodology behind detecting AI-generated code submissions:

- **Perplexity Analysis**: Utilizes the GPT-2 language model to calculate the perplexity of code snippets. Lower perplexity values often indicate AI-generated code due to the model's familiarity with such patterns.
- **Pattern Recognition**: Analyzes code structure, variable naming conventions, comment ratios, and indentation consistency to identify characteristics typical of AI-generated code.
- **Caching for Speed**: Implements LRU caching to accelerate repeated analysis and improve scalability for large datasets.
- **Singleton Model Loading**: Ensures models are loaded only once per session, reducing memory usage and initialization time.
- **Batch and Parallel Processing**: Supports efficient batch analysis and parallel execution for rapid review of multiple code samples.

> This modular AI detection logic is designed for integration into automated code review systems, online judges, and educational platforms to help maintain code authenticity and integrity.

In [8]:
class OptimizedAIGeneratedCodeDetector:
    """
    Enhanced AI detection with AST features + Perplexity
    No training required - uses pre-trained GPT-2 + rule-based AST parsing
    """
    
    _instance = None
    
    def __new__(cls):
        """Singleton pattern - load models only once"""
        if cls._instance is None:
            cls._instance = super().__new__(cls)
            cls._instance._initialized = False
        return cls._instance
    
    def __init__(self):
        if self._initialized:
            return
            
        # Load GPT-2 pre-trained model (no training needed)
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        print(f"Loading GPT-2 model on {self.device}...")
        
        self.gpt2_model = GPT2LMHeadModel.from_pretrained('gpt2').to(self.device)
        self.gpt2_tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
        self.gpt2_tokenizer.pad_token = self.gpt2_tokenizer.eos_token
        
        # Set to eval mode for faster inference
        self.gpt2_model.eval()
        
        self._initialized = True
        print("✓ GPT-2 model loaded successfully (pre-trained, no training required)!")
    
    @lru_cache(maxsize=1000)
    def calculate_perplexity(self, code):
        """
        Cached perplexity calculation using pre-trained GPT-2
        Low perplexity = AI-likely (predictable), High = Human-likely (variable)
        """
        try:
            code_lines = [line.strip() for line in code.split('\n') if line.strip()]
            code_text = ' '.join(code_lines)
            
            encodings = self.gpt2_tokenizer(
                code_text, 
                return_tensors='pt', 
                truncation=True, 
                max_length=512
            ).to(self.device)
            
            with torch.no_grad():
                outputs = self.gpt2_model(**encodings, labels=encodings['input_ids'])
                perplexity = torch.exp(outputs.loss)
                
            return float(perplexity)
        except:
            return float('inf')
    
    @lru_cache(maxsize=2000)
    def extract_ast_features(self, code):
        """
        Extract structural features using AST (rule-based, no training)
        AI code shows different structural patterns than human code
        """
        features = {}
        
        try:
            tree = ast.parse(code)
            
            # 1. Node Type Distribution
            node_types = [type(node).__name__ for node in ast.walk(tree)]
            total_nodes = len(node_types)
            features['total_nodes'] = total_nodes
            features['unique_node_ratio'] = len(set(node_types)) / max(total_nodes, 1)
            
            # 2. Depth Analysis (AI tends to have uniform depth)
            def get_depths(node, depth=0):
                depths = [depth]
                for child in ast.iter_child_nodes(node):
                    depths.extend(get_depths(child, depth + 1))
                return depths
            
            depths = get_depths(tree)
            features['max_depth'] = max(depths) if depths else 0
            features['avg_depth'] = np.mean(depths) if depths else 0
            features['depth_variance'] = np.var(depths) if len(depths) > 1 else 0
            
            # 3. Control Flow Complexity
            num_functions = sum(1 for _ in ast.walk(tree) if isinstance(_, ast.FunctionDef))
            num_conditions = sum(1 for _ in ast.walk(tree) if isinstance(_, ast.If))
            num_loops = sum(1 for _ in ast.walk(tree) if isinstance(_, (ast.For, ast.While)))
            num_try = sum(1 for _ in ast.walk(tree) if isinstance(_, ast.Try))
            
            features['num_functions'] = num_functions
            features['num_conditions'] = num_conditions
            features['num_loops'] = num_loops
            features['num_try_except'] = num_try
            features['cyclomatic_complexity'] = 1 + num_conditions + num_loops
            
            # 4. AI-Specific Patterns
            # AI code tends to be "too perfect" - balanced structure, low variance
            features['is_perfectly_balanced'] = (
                features['depth_variance'] < 0.8 and 
                features['cyclomatic_complexity'] < 4 and
                features['max_depth'] < 6
            )
            
            # AI code rarely has try-except (error handling)
            features['has_error_handling'] = num_try > 0
            
            # 5. Complexity Ratio (AI avoids very simple or very complex)
            if total_nodes > 0:
                features['complexity_ratio'] = features['cyclomatic_complexity'] / total_nodes
            else:
                features['complexity_ratio'] = 0
            
            # 6. Pattern Repetition (AI often repeats similar structures)
            node_type_counts = {}
            for node_type in node_types:
                node_type_counts[node_type] = node_type_counts.get(node_type, 0) + 1
            
            if node_type_counts:
                max_repetition = max(node_type_counts.values())
                features['max_node_repetition'] = max_repetition / total_nodes
            else:
                features['max_node_repetition'] = 0
            
            # 7. Parse Success (human code may have minor issues)
            features['parse_success'] = True
            
        except SyntaxError:
            # Human code more likely to have syntax quirks
            features['parse_success'] = False
            features['parse_error'] = True
        except Exception as e:
            features['parse_success'] = False
            features['parse_error'] = True
        
        return tuple(features.items())  # Return tuple for caching
    
    @lru_cache(maxsize=2000)
    def analyze_code_patterns(self, code):
        """
        Fast pattern analysis (regex-based, no training)
        """
        features = {}
        
        lines = code.split('\n')
        total_lines = len(lines)
        
        # 1. Comment Analysis (AI often over-comments or perfectly comments)
        comment_lines = sum(1 for line in lines if line.strip().startswith('#'))
        docstring_count = len(re.findall(r'"""[\s\S]*?"""|\'\'\'[\s\S]*?\'\'\'', code))
        features['comment_ratio'] = comment_lines / max(total_lines, 1)
        features['has_excessive_comments'] = features['comment_ratio'] > 0.4
        features['has_docstrings'] = docstring_count > 0
        
        # 2. Variable Naming Patterns
        try:
            var_pattern = r'\b[a-zA-Z_][a-zA-Z0-9_]*\b'
            var_names = re.findall(var_pattern, code)
            keywords = {'def', 'class', 'if', 'else', 'for', 'while', 'return', 'import', 
                       'from', 'try', 'except', 'with', 'as', 'in', 'and', 'or', 'not'}
            var_names = [v for v in var_names if v not in keywords]
            
            if var_names:
                avg_name_length = np.mean([len(name) for name in var_names])
                # AI prefers descriptive names (8+ chars)
                descriptive_names = sum(1 for name in var_names if len(name) >= 8)
                # AI often uses underscores consistently
                underscore_names = sum(1 for name in var_names if '_' in name)
                
                features['avg_var_name_length'] = avg_name_length
                features['descriptive_name_ratio'] = descriptive_names / len(var_names)
                features['underscore_ratio'] = underscore_names / len(var_names)
            else:
                features['avg_var_name_length'] = 0
                features['descriptive_name_ratio'] = 0
                features['underscore_ratio'] = 0
        except:
            features['avg_var_name_length'] = 0
            features['descriptive_name_ratio'] = 0
            features['underscore_ratio'] = 0
        
        # 3. Indentation Consistency (AI is always perfect)
        indents = [len(line) - len(line.lstrip()) for line in lines if line.strip()]
        if indents:
            features['indent_consistency'] = 1 - (np.std(indents) / (np.mean(indents) + 1))
        else:
            features['indent_consistency'] = 0
        
        # 4. Line Length Uniformity (AI tends to have uniform line lengths)
        line_lengths = [len(line) for line in lines if line.strip()]
        if line_lengths:
            features['line_length_variance'] = np.var(line_lengths)
        else:
            features['line_length_variance'] = 0
        
        return tuple(features.items())  # Return tuple for caching
    
    def detect_ai_generated(self, code):
        """Fixed scoring with research-backed thresholds"""
        
        perplexity = self.calculate_perplexity(code)
        ast_features_tuple = self.extract_ast_features(code)
        pattern_features_tuple = self.analyze_code_patterns(code)
        
        ast_features = dict(ast_features_tuple)
        pattern_features = dict(pattern_features_tuple)
        
        ai_score = 0.0
        score_breakdown = {}
        
        # --- FIXED PERPLEXITY SCORING (40% weight, increased) ---
        if perplexity < 10:
            perplexity_score = 0.40  # VERY AI-like (your case: 5.36)
        elif perplexity < 30:
            perplexity_score = 0.30  # Strong AI signal
        elif perplexity < 60:
            perplexity_score = 0.20  # Moderate
        elif perplexity < 100:
            perplexity_score = 0.10  # Low suspicion
        else:
            perplexity_score = 0.0   # Human-like (>100)
        
        ai_score += perplexity_score
        score_breakdown['perplexity'] = perplexity_score
        
        # --- AST FEATURES (35% weight, decreased) ---
        ast_score = 0.0
        
        if not ast_features.get('parse_error', False):
            if ast_features.get('parse_success', False):
                ast_score += 0.03
            if ast_features.get('is_perfectly_balanced', False):
                ast_score += 0.15
            if ast_features.get('depth_variance', 1.0) < 1.0:
                ast_score += 0.08
            if ast_features.get('unique_node_ratio', 1.0) < 0.4:
                ast_score += 0.08
            if not ast_features.get('has_error_handling', False):
                ast_score += 0.03
        
        ai_score += min(ast_score, 0.35)
        score_breakdown['ast'] = min(ast_score, 0.35)
        
        # --- PATTERN SCORING (25% weight, decreased) ---
        pattern_score = 0.0
        
        if pattern_features.get('has_excessive_comments', False):
            pattern_score += 0.10
        if pattern_features.get('avg_var_name_length', 0) > 10:
            pattern_score += 0.06
        if pattern_features.get('underscore_ratio', 0) > 0.7:
            pattern_score += 0.04
        if pattern_features.get('indent_consistency', 0) > 0.95:
            pattern_score += 0.05
        
        ai_score += min(pattern_score, 0.25)
        score_breakdown['pattern'] = min(pattern_score, 0.25)
        
        # === FIXED THRESHOLDS ===
        ai_probability = min(ai_score, 1.0)
        
        if ai_probability >= 0.70:  # Lowered from 0.75
            verdict = "HIGH CONFIDENCE: Code is very likely AI-generated"
            risk_level = "HIGH"
        elif ai_probability >= 0.55:  # Lowered from 0.60
            verdict = "MODERATE CONFIDENCE: Code shows significant AI patterns"
            risk_level = "MEDIUM"
        elif ai_probability >= 0.35:  # Lowered from 0.40
            verdict = "LOW CONFIDENCE: Some AI-like patterns detected"
            risk_level = "LOW"
        else:
            verdict = "CLEAN: Code appears human-written"
            risk_level = "NONE"
        
        return {
            'ai_probability': round(ai_probability, 3),
            'is_ai_generated': ai_probability >= 0.55,  # Lowered threshold
            'risk_level': risk_level,
            'verdict': verdict,
            'score_breakdown': {
                'perplexity_score': round(score_breakdown['perplexity'], 3),
                'ast_score': round(score_breakdown['ast'], 3),
                'pattern_score': round(score_breakdown['pattern'], 3)
            },
            'metrics': {
                'perplexity': round(perplexity, 2),
                'ast_features': ast_features,
                'pattern_features': pattern_features
            }
        }


---

## Plagiarism Detection Logic

This section details the approach used for detecting plagiarism in code submissions:

- **Common Algorithm Patterns**: The detector checks submitted code against a set of well-known algorithmic patterns (e.g., quicksort, binary search, bubble sort, merge sort, recursive Fibonacci).
- **Hash-Based Matching**: Each pattern is pre-processed and stored as a hash for fast exact matching. If a code snippet matches a known pattern hash, it is flagged as an exact match.
- **Similarity Analysis**: For non-exact matches, the detector uses sequence similarity to compare code structure and logic, flagging submissions with high similarity scores.
- **Source Attribution**: When plagiarism is detected, the system reports the sources (e.g., StackOverflow, GitHub, LeetCode) where the matching pattern is commonly found.
- **Performance**: The logic is optimized for speed using caching and early termination, making it suitable for large-scale automated code review.

> This modular approach allows for easy extension with new patterns and sources, and can be integrated into backend services for real-time plagiarism detection.

In [3]:
class OptimizedPlagiarismDetector:
    """
    Optimized plagiarism detection against common patterns
    No training required - uses hash-based matching + similarity
    """
    
    def __init__(self):
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        
        # Pre-compute hashes for faster lookup
        self.pattern_hashes = {}
        self.patterns_by_hash = {}
        self._load_common_snippets()
        
    def _load_common_snippets(self):
        """Load common algorithm patterns (expandable database)"""
        snippets = {
            'quicksort_basic': {
                'pattern': 'def quicksort(arr): if len(arr) <= 1: return arr',
                'sources': ['stackoverflow', 'github']
            },
            'fibonacci_recursive': {
                'pattern': 'def fibonacci(n): if n <= 1: return n return fibonacci(n-1) + fibonacci(n-2)',
                'sources': ['common_algorithm']
            },
            'binary_search': {
                'pattern': 'def binary_search(arr, target): left = 0 right = len(arr) - 1',
                'sources': ['leetcode', 'github']
            },
            'bubble_sort': {
                'pattern': 'def bubble_sort(arr): for i in range(len(arr)): for j in range(len(arr)-i-1):',
                'sources': ['stackoverflow']
            },
            'merge_sort': {
                'pattern': 'def merge_sort(arr): if len(arr) > 1: mid = len(arr) // 2',
                'sources': ['github', 'common_algorithm']
            }
        }
        
        # Pre-compute all hashes
        for name, info in snippets.items():
            pattern_hash = hashlib.md5(info['pattern'].encode()).hexdigest()
            self.pattern_hashes[name] = pattern_hash
            self.patterns_by_hash[pattern_hash] = {
                'name': name,
                'pattern': info['pattern'],
                'sources': info['sources']
            }
    
    @lru_cache(maxsize=2000)
    def normalize_for_comparison(self, code):
        """Fast normalization for comparison"""
        code = re.sub(r'#.*', '', code)
        code = re.sub(r'"""[\s\S]*?"""|\'\'\'[\s\S]*?\'\'\'', '', code)
        code = re.sub(r'\s+', ' ', code).strip()
        return code
    
    def check_against_common_patterns(self, code):
        """Check against known patterns"""
        normalized_code = self.normalize_for_comparison(code)
        code_hash = hashlib.md5(normalized_code.encode()).hexdigest()
        
        # Fast exact match check
        if code_hash in self.patterns_by_hash:
            pattern_info = self.patterns_by_hash[code_hash]
            return [{
                'pattern': pattern_info['name'],
                'similarity': 1.0,
                'sources': pattern_info['sources'],
                'match_type': 'exact'
            }]
        
        # Similarity check
        matches = []
        for pattern_hash, pattern_info in self.patterns_by_hash.items():
            len_diff = abs(len(normalized_code) - len(pattern_info['pattern']))
            if len_diff / max(len(normalized_code), len(pattern_info['pattern'])) > 0.3:
                continue
            
            similarity = difflib.SequenceMatcher(
                None, 
                normalized_code, 
                pattern_info['pattern']
            ).ratio()
            
            if similarity > 0.8:
                matches.append({
                    'pattern': pattern_info['name'],
                    'similarity': similarity,
                    'sources': pattern_info['sources'],
                    'match_type': 'similar'
                })
        
        return matches
    
    def detect_online_plagiarism(self, code):
        """Detect plagiarism from known sources"""
        matches = self.check_against_common_patterns(code)
        
        if matches:
            max_similarity = max(match['similarity'] for match in matches)
            return {
                'is_plagiarized': max_similarity > 0.85,
                'max_similarity': max_similarity,
                'matches': matches,
                'sources_found': list(set(source for match in matches for source in match['sources']))
            }
        
        return {
            'is_plagiarized': False,
            'max_similarity': 0.0,
            'matches': [],
            'sources_found': []
        }

---

## Code Analysis and Recommendation Engine

This section introduces the integrated code analysis engine, which combines AI-generated code detection and plagiarism detection to provide a comprehensive assessment of code submissions:

- **Parallel Processing**: The engine runs AI and plagiarism checks in parallel for faster results, making it suitable for batch analysis and large datasets.
- **Suspiciousness Scoring**: Each code snippet is evaluated for signs of AI generation and plagiarism, with an overall suspiciousness score and detailed breakdown.
- **Actionable Recommendations**: Based on the analysis, the engine provides clear recommendations, such as flagging high-risk submissions, suggesting manual review, or confirming clean code.
- **Batch Support**: Multiple code samples can be analyzed simultaneously, with results aggregated for efficient review.
- **Cache Management**: Built-in cache clearing methods help manage memory and maintain performance during repeated or large-scale analysis.

> This modular recommendation engine can be integrated into automated code review systems, online judges, or educational platforms to enhance code integrity and fairness.

In [4]:
class OptimizedCodeChecker:
    """
    Production-ready code checker with parallel processing
    No training required - uses pre-trained models + rule-based analysis
    """
    
    def __init__(self):
        self.ai_detector = OptimizedAIGeneratedCodeDetector()
        self.plagiarism_detector = OptimizedPlagiarismDetector()
        self.executor = ThreadPoolExecutor(max_workers=2)
    
    def analyze_code(self, code, parallel=True):
        """
        Analyze single code submission
        
        Args:
            code: User-submitted code string
            parallel: Run AI and plagiarism detection in parallel
        
        Returns:
            Comprehensive analysis dict
        """
        
        if parallel:
            # Parallel execution for speed
            future_ai = self.executor.submit(self.ai_detector.detect_ai_generated, code)
            future_plag = self.executor.submit(self.plagiarism_detector.detect_online_plagiarism, code)
            
            ai_result = future_ai.result()
            plagiarism_result = future_plag.result()
        else:
            ai_result = self.ai_detector.detect_ai_generated(code)
            plagiarism_result = self.plagiarism_detector.detect_online_plagiarism(code)
        
        overall_suspicious = (
            ai_result['is_ai_generated'] or 
            plagiarism_result['is_plagiarized']
        )
        
        return {
            'overall_suspicious': overall_suspicious,
            'ai_detection': ai_result,
            'plagiarism_detection': plagiarism_result,
            'recommendation': self._get_recommendation(ai_result, plagiarism_result)
        }
    
    def analyze_batch(self, codes):
        """Batch analysis for multiple submissions"""
        results = []
        
        with ThreadPoolExecutor(max_workers=4) as executor:
            futures = [executor.submit(self.analyze_code, code, False) for code in codes]
            results = [future.result() for future in futures]
        
        return results
    
    def _get_recommendation(self, ai_result, plagiarism_result):
        """Generate actionable recommendation"""
        if ai_result['is_ai_generated'] and plagiarism_result['is_plagiarized']:
            return "🚨 CRITICAL: Code shows BOTH AI generation and plagiarism patterns"
        elif ai_result['is_ai_generated']:
            prob = ai_result['ai_probability']
            return f"⚠️  AI DETECTED: Code likely AI-generated (confidence: {prob:.1%})"
        elif plagiarism_result['is_plagiarized']:
            sources = ', '.join(plagiarism_result['sources_found'])
            return f"⚠️  PLAGIARISM: Matches known sources ({sources})"
        elif ai_result['ai_probability'] > 0.4 or plagiarism_result['max_similarity'] > 0.6:
            return "⚡ MODERATE RISK: Some suspicious patterns, review recommended"
        else:
            return "✅ CLEAN: No significant AI or plagiarism detected"
    
    def clear_cache(self):
        """Clear LRU caches to free memory"""
        self.ai_detector.calculate_perplexity.cache_clear()
        self.ai_detector.extract_ast_features.cache_clear()
        self.ai_detector.analyze_code_patterns.cache_clear()
        self.plagiarism_detector.normalize_for_comparison.cache_clear()

---

## Test Cases and Performance Metrics

This section provides a suite of test cases to validate the code analysis engine and benchmark its performance:

- **Test Coverage**: Includes examples of suspicious AI-like code, common algorithms (potential plagiarism), and original code to demonstrate detection capabilities.
- **Single and Batch Analysis**: Measures the time taken for individual and batch code analysis, highlighting the speedup from parallel processing.
- **Performance Reporting**: Outputs suspiciousness, AI probability, plagiarism status, and recommendations for each test case, along with timing metrics.
- **Cache Efficiency**: (Commented) Optionally tests cache performance for repeated analysis, showing the benefits of caching in large-scale scenarios.

> Use these tests to ensure the reliability and efficiency of the detection pipeline before deploying in production or integrating with automated review systems.

In [9]:
# TESTING & DEMONSTRATION
# ============================================================================

def test_optimized_checker():
    """Comprehensive test with performance metrics"""
    import time
    
    checker = OptimizedCodeChecker()
    
    test_codes = [
        {
            'name': '🤖 Suspicious AI-like code',
            'code': '''
def calculate_fibonacci_sequence(number_of_terms):
    """
    Calculate fibonacci sequence up to n terms.
    This function uses iteration to calculate fibonacci numbers efficiently.
    """
    if number_of_terms <= 0:
        return "Please provide a positive integer"
    elif number_of_terms == 1:
        return [0]
    elif number_of_terms == 2:
        return [0, 1]
    else:
        fibonacci_sequence = [0, 1]
        for current_index in range(2, number_of_terms):
            next_number = fibonacci_sequence[current_index - 1] + fibonacci_sequence[current_index - 2]
            fibonacci_sequence.append(next_number)
        return fibonacci_sequence
            '''
        },
        {
            'name': '📋 Common algorithm (plagiarism suspect)',
            'code': '''
def fibonacci(n):
    if n <= 1:
        return n
    return fibonacci(n-1) + fibonacci(n-2)
            '''
        },
        {
            'name': '👤 Human-written code',
            'code': '''
def calc(x, y):
    # quick calc
    z = x + y
    return z * 2
            '''
        },
        {
            'name': '🧪 Complex human code',
            'code': '''
def solve(arr):
    res = []
    for i in range(len(arr)):
        if arr[i] % 2:
            res.append(arr[i])
    return res
            '''
        }
    ]
    
    print("=" * 80)
    print("🔍 COMPREHENSIVE CODE ANALYSIS TEST")
    print("=" * 80)
    
    # Single analysis
    print("\n📊 Testing single code analysis...\n")
    start = time.time()
    
    for test in test_codes:
        print(f"\n{'─' * 80}")
        print(f"Testing: {test['name']}")
        print(f"{'─' * 80}")
        
        result = checker.analyze_code(test['code'])
        
        print(f"\n🎯 VERDICT: {result['recommendation']}")
        print(f"   Overall Suspicious: {result['overall_suspicious']}")
        
        ai = result['ai_detection']
        print(f"\n🤖 AI Detection:")
        print(f"   Probability: {ai['ai_probability']:.1%}")
        print(f"   Risk Level: {ai['risk_level']}")
        print(f"   Perplexity: {ai['metrics']['perplexity']:.2f}")
        print(f"   Score Breakdown:")
        print(f"      - Perplexity: {ai['score_breakdown']['perplexity_score']:.3f}")
        print(f"      - AST: {ai['score_breakdown']['ast_score']:.3f}")
        print(f"      - Pattern: {ai['score_breakdown']['pattern_score']:.3f}")
        
        plag = result['plagiarism_detection']
        print(f"\n📋 Plagiarism Detection:")
        print(f"   Is Plagiarized: {plag['is_plagiarized']}")
        print(f"   Max Similarity: {plag['max_similarity']:.1%}")
        if plag['sources_found']:
            print(f"   Sources: {', '.join(plag['sources_found'])}")
    
    single_time = time.time() - start
    print(f"\n⏱️  Single analysis time: {single_time:.2f}s")
    
    # Batch analysis
    print(f"\n{'=' * 80}")
    print("🚀 Testing batch analysis (parallel processing)...")
    print(f"{'=' * 80}")
    
    start = time.time()
    batch_results = checker.analyze_batch([test['code'] for test in test_codes])
    batch_time = time.time() - start
    
    print(f"✓ Processed {len(test_codes)} codes in {batch_time:.2f}s")
    print(f"✓ Speedup: {single_time/batch_time:.2f}x faster")
    
    # Cache test
    print(f"\n{'=' * 80}")
    print("💾 Testing cache performance (re-analyzing same code)...")
    print(f"{'=' * 80}")
    
    start = time.time()
    for _ in range(5):
        checker.analyze_code(test_codes[0]['code'])
    cached_time = time.time() - start
    
    print(f"✓ 5 cached analyses: {cached_time:.3f}s ({cached_time/5:.3f}s per analysis)")
    #print(f"✓ Cache speedup: {(single_time/len(test_codes))/(cached_time/5):.1f}x faster")
    
    print(f"\n{'=' * 80}")
    print("✅ ALL TESTS COMPLETED")
    print(f"{'=' * 80}\n")


if __name__ == "__main__":
    print("\n" + "=" * 80)
    print("  PRODUCTION-READY AI CODE DETECTOR")
    print("  No Training Required | Pre-trained GPT-2 + AST Analysis")
    print("=" * 80 + "\n")
    
    test_optimized_checker()


  PRODUCTION-READY AI CODE DETECTOR
  No Training Required | Pre-trained GPT-2 + AST Analysis

Loading GPT-2 model on cuda...
✓ GPT-2 model loaded successfully (pre-trained, no training required)!
🔍 COMPREHENSIVE CODE ANALYSIS TEST

📊 Testing single code analysis...


────────────────────────────────────────────────────────────────────────────────
Testing: 🤖 Suspicious AI-like code
────────────────────────────────────────────────────────────────────────────────

🎯 VERDICT: ⚡ MODERATE RISK: Some suspicious patterns, review recommended
   Overall Suspicious: False

🤖 AI Detection:
   Probability: 54.0%
   Risk Level: LOW
   Perplexity: 5.36
   Score Breakdown:
      - Perplexity: 0.400
      - AST: 0.140
      - Pattern: 0.000

📋 Plagiarism Detection:
   Is Plagiarized: False
   Max Similarity: 0.0%

────────────────────────────────────────────────────────────────────────────────
Testing: 📋 Common algorithm (plagiarism suspect)
───────────────────────────────────────────────────────────

---

## Model Saving and Deployment

This section demonstrates how to save the optimized GPT-2 model and tokenizer in HuggingFace format for future use or production deployment. Saving models in this way allows for easy loading, sharing, and integration into backend services or cloud environments.

- **save_dir**: The directory where the model and tokenizer will be stored.
- **gpt2_model.save_pretrained(save_dir)**: Saves the model weights and configuration.
- **gpt2_tokenizer.save_pretrained(save_dir)**: Saves the tokenizer files for consistent preprocessing.

> After saving, you can reload the model and tokenizer using `from_pretrained(save_dir)` in any compatible environment.

**Tip:** Always version your saved models and document the training or fine-tuning process for reproducibility and auditability.

In [ ]:
# Save the optimized model and tokenizer
save_dir = "../models"                 # choose a folder name
gpt2_model = GPT2LMHeadModel.from_pretrained("gpt2")
gpt2_tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

# save everything in Hugging-Face format
gpt2_model.save_pretrained(save_dir)
gpt2_tokenizer.save_pretrained(save_dir)

print(f"Model and tokenizer saved in {save_dir}")


Model and tokenizer saved in ../models


In [2]:
import numpy as np
import torch
import re
import ast
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import warnings
warnings.filterwarnings('ignore')


class OptimizedAIGeneratedCodeDetector:
    """
    AI detection with conflict resolution between perplexity and AST
    """
    
    _instance = None
    
    def __new__(cls):
        if cls._instance is None:
            cls._instance = super().__new__(cls)
            cls._instance._initialized = False
        return cls._instance
    
    def __init__(self):
        if self._initialized:
            return
            
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        print(f"Loading GPT-2 model on {self.device}...")
        
        self.gpt2_model = GPT2LMHeadModel.from_pretrained('gpt2').to(self.device)
        self.gpt2_tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
        self.gpt2_tokenizer.pad_token = self.gpt2_tokenizer.eos_token
        self.gpt2_model.eval()
        
        self._initialized = True
        print("✓ GPT-2 model loaded successfully!\n")
    
    def calculate_perplexity(self, code):
        """Calculate perplexity - LOW = AI-likely"""
        try:
            code_lines = [line.strip() for line in code.split('\n') if line.strip()]
            code_text = ' '.join(code_lines)
            
            encodings = self.gpt2_tokenizer(
                code_text, 
                return_tensors='pt', 
                truncation=True, 
                max_length=512
            ).to(self.device)
            
            with torch.no_grad():
                outputs = self.gpt2_model(**encodings, labels=encodings['input_ids'])
                perplexity = torch.exp(outputs.loss)
            
            return float(perplexity)
            
        except Exception as e:
            print(f"[ERROR] Perplexity calculation failed: {e}")
            return 150.0
    
    def extract_ast_features(self, code):
        """Extract AST features"""
        features = {}
        
        try:
            tree = ast.parse(code)
            
            # Node analysis
            node_types = [type(node).__name__ for node in ast.walk(tree)]
            total_nodes = len(node_types)
            features['total_nodes'] = total_nodes
            features['unique_node_ratio'] = len(set(node_types)) / max(total_nodes, 1)
            
            # Depth analysis
            def get_depths(node, depth=0):
                depths = [depth]
                for child in ast.iter_child_nodes(node):
                    depths.extend(get_depths(child, depth + 1))
                return depths
            
            depths = get_depths(tree)
            features['max_depth'] = max(depths) if depths else 0
            features['depth_variance'] = np.var(depths) if len(depths) > 1 else 0
            
            # Control flow
            num_conditions = sum(1 for _ in ast.walk(tree) if isinstance(_, ast.If))
            num_loops = sum(1 for _ in ast.walk(tree) if isinstance(_, (ast.For, ast.While)))
            num_try = sum(1 for _ in ast.walk(tree) if isinstance(_, ast.Try))
            
            features['num_conditions'] = num_conditions
            features['num_loops'] = num_loops
            features['cyclomatic_complexity'] = 1 + num_conditions + num_loops
            
            # AI patterns (perfect balance, low variance)
            features['is_perfectly_balanced'] = (
                features['depth_variance'] < 1.0 and 
                features['cyclomatic_complexity'] < 4
            )
            features['has_error_handling'] = num_try > 0
            features['parse_success'] = True
            
        except:
            features['parse_success'] = False
            features['parse_error'] = True
        
        return features
    
    def analyze_code_patterns(self, code):
        """Pattern analysis"""
        features = {}
        
        lines = code.split('\n')
        total_lines = len(lines)
        
        # Comments
        comment_lines = sum(1 for line in lines if line.strip().startswith('#'))
        docstring_count = len(re.findall(r'"""[\s\S]*?"""|\'\'\'[\s\S]*?\'\'\'', code))
        features['comment_ratio'] = comment_lines / max(total_lines, 1)
        features['has_docstrings'] = docstring_count > 0
        
        # Variable naming
        try:
            var_pattern = r'\b[a-zA-Z_][a-zA-Z0-9_]*\b'
            var_names = re.findall(var_pattern, code)
            keywords = {'def', 'class', 'if', 'else', 'for', 'while', 'return', 'import'}
            var_names = [v for v in var_names if v not in keywords]
            
            if var_names:
                avg_name_length = np.mean([len(name) for name in var_names])
                long_names = sum(1 for name in var_names if len(name) >= 10)
                
                features['avg_var_name_length'] = avg_name_length
                features['long_name_ratio'] = long_names / len(var_names)
            else:
                features['avg_var_name_length'] = 0
                features['long_name_ratio'] = 0
        except:
            features['avg_var_name_length'] = 0
            features['long_name_ratio'] = 0
        
        # Indentation
        indents = [len(line) - len(line.lstrip()) for line in lines if line.strip()]
        if indents:
            features['indent_consistency'] = 1 - (np.std(indents) / (np.mean(indents) + 1))
        else:
            features['indent_consistency'] = 0
        
        return features
    
    def detect_ai_generated(self, code):
        """
        FIXED: Perplexity-first approach with AST as secondary
        
        Key insight: Perplexity is the STRONGEST indicator for AI detection
        AST features can have false negatives, so we weight perplexity higher
        """
        
        # Calculate features
        perplexity = self.calculate_perplexity(code)
        ast_features = self.extract_ast_features(code)
        pattern_features = self.analyze_code_patterns(code)
        
        print(f"\n{'='*70}")
        print(f"DETAILED ANALYSIS:")
        print(f"{'='*70}")
        
        # ================================================================
        # PERPLEXITY ANALYSIS (PRIMARY INDICATOR - 60% weight)
        # Research: Most reliable single metric for AI detection
        # ================================================================
        
        print(f"\n1️⃣  PERPLEXITY ANALYSIS")
        print(f"    Value: {perplexity:.2f}")
        
        if perplexity < 10:
            perplexity_score = 0.60
            perplexity_level = "CRITICAL"
            interpretation = "EXTREMELY LOW - Very strong AI signature"
        elif perplexity < 20:
            perplexity_score = 0.50
            perplexity_level = "HIGH"
            interpretation = "Very low - Strong AI indicator"
        elif perplexity < 35:
            perplexity_score = 0.35
            perplexity_level = "MODERATE-HIGH"
            interpretation = "Low - Significant AI suspicion"
        elif perplexity < 60:
            perplexity_score = 0.20
            perplexity_level = "MODERATE"
            interpretation = "Moderate - Some AI patterns"
        elif perplexity < 100:
            perplexity_score = 0.10
            perplexity_level = "LOW"
            interpretation = "Higher - Leaning human"
        else:
            perplexity_score = 0.0
            perplexity_level = "NONE"
            interpretation = "High - Human-like variability"
        
        print(f"    Level: {perplexity_level}")
        print(f"    {interpretation}")
        print(f"    Score Contribution: {perplexity_score:.3f} / 0.600")
        
        # ================================================================
        # AST STRUCTURE ANALYSIS (SECONDARY - 25% weight)
        # Can have false negatives, so lower weight
        # ================================================================
        
        print(f"\n2️⃣  AST STRUCTURE ANALYSIS")
        print(f"    Balanced: {ast_features.get('is_perfectly_balanced', False)}")
        print(f"    Depth Variance: {ast_features.get('depth_variance', 0):.2f}")
        print(f"    Complexity: {ast_features.get('cyclomatic_complexity', 0)}")
        
        ast_score = 0.0
        
        if not ast_features.get('parse_error', False):
            # Perfect balance (strong AI indicator)
            if ast_features.get('is_perfectly_balanced', False):
                ast_score += 0.15
                print(f"    ✓ Perfect balance detected (+0.15)")
            
            # Low depth variance (AI pattern)
            if ast_features.get('depth_variance', 10) < 1.5:
                ast_score += 0.10
                print(f"    ✓ Low depth variance (+0.10)")
            
            # Very low node diversity
            if ast_features.get('unique_node_ratio', 1.0) < 0.35:
                ast_score += 0.05
        
        ast_score = min(ast_score, 0.25)
        print(f"    Score Contribution: {ast_score:.3f} / 0.250")
        
        # ================================================================
        # PATTERN ANALYSIS (TERTIARY - 15% weight)
        # ================================================================
        
        print(f"\n3️⃣  PATTERN ANALYSIS")
        print(f"    Avg Variable Length: {pattern_features.get('avg_var_name_length', 0):.1f}")
        print(f"    Comment Ratio: {pattern_features.get('comment_ratio', 0):.2f}")
        print(f"    Indent Consistency: {pattern_features.get('indent_consistency', 0):.2f}")
        
        pattern_score = 0.0
        
        # Very long descriptive names (AI loves these)
        if pattern_features.get('avg_var_name_length', 0) > 15:
            pattern_score += 0.08
            print(f"    ✓ Very long variable names (+0.08)")
        elif pattern_features.get('avg_var_name_length', 0) > 10:
            pattern_score += 0.04
        
        # Excessive documentation
        if pattern_features.get('has_docstrings', False) and pattern_features.get('comment_ratio', 0) > 0.3:
            pattern_score += 0.07
            print(f"    ✓ Excessive documentation (+0.07)")
        
        # Perfect indentation
        if pattern_features.get('indent_consistency', 0) > 0.98:
            pattern_score += 0.05
        
        pattern_score = min(pattern_score, 0.15)
        print(f"    Score Contribution: {pattern_score:.3f} / 0.150")
        
        # ================================================================
        # FINAL SCORING
        # ================================================================
        
        total_score = perplexity_score + ast_score + pattern_score
        ai_probability = min(total_score, 1.0)
        
        print(f"\n{'='*70}")
        print(f"TOTAL AI PROBABILITY: {ai_probability:.3f} ({ai_probability*100:.1f}%)")
        print(f"{'='*70}")
        
        # Determine verdict
        if ai_probability >= 0.65:
            verdict = "HIGH CONFIDENCE: Code is very likely AI-generated"
            risk_level = "HIGH"
        elif ai_probability >= 0.45:
            verdict = "MODERATE CONFIDENCE: Code shows significant AI patterns"
            risk_level = "MEDIUM"
        elif ai_probability >= 0.30:
            verdict = "LOW CONFIDENCE: Some AI-like patterns detected"
            risk_level = "LOW"
        else:
            verdict = "CLEAN: Code appears human-written"
            risk_level = "NONE"
        
        # Conflict detection
        conflict_detected = False
        if perplexity < 15 and ast_score < 0.10:
            conflict_detected = True
            verdict += " [⚠️  CONFLICT: Low perplexity but human-like structure]"
            print(f"\n⚠️  CONFLICT DETECTED:")
            print(f"    Perplexity signals strong AI ({perplexity:.2f})")
            print(f"    But AST structure appears human-like")
            print(f"    This may indicate: Common code patterns, or AI with unusual structure")
        
        return {
            'ai_probability': round(ai_probability, 3),
            'is_ai_generated': ai_probability >= 0.45,
            'risk_level': risk_level,
            'verdict': verdict,
            'conflict_detected': conflict_detected,
            'score_breakdown': {
                'perplexity_score': round(perplexity_score, 3),
                'ast_score': round(ast_score, 3),
                'pattern_score': round(pattern_score, 3)
            },
            'metrics': {
                'perplexity': round(perplexity, 2),
                'perplexity_level': perplexity_level,
                'ast_features': ast_features,
                'pattern_features': pattern_features
            }
        }


# Simple test
if __name__ == "__main__":
    detector = OptimizedAIGeneratedCodeDetector()
    
    test_code = '''
def calculate_fibonacci_sequence(number_of_terms):
        fibonacci_sequence = [0, 1]
        for current_index in range(2, number_of_terms):
            next_number = fibonacci_sequence[current_index - 1] + fibonacci_sequence[current_index - 2]
            fibonacci_sequence.append(next_number)
        return fibonacci_sequence
    '''
    
    print("="*70)
    print("TESTING: AI-like Fibonacci Code")
    print("="*70)
    
    result = detector.detect_ai_generated(test_code)
    
    print(f"\n\n{'🎯 FINAL VERDICT':^70}")
    print(f"{'='*70}")
    print(f"Verdict: {result['verdict']}")
    print(f"AI Probability: {result['ai_probability']:.1%}")
    print(f"Risk Level: {result['risk_level']}")
    print(f"Is AI Generated: {result['is_ai_generated']}")
    print(f"Conflict Detected: {result['conflict_detected']}")
    print(f"{'='*70}")

Loading GPT-2 model on cuda...


RuntimeError: CUDA error: unknown error
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [ ]:
# Add this to your test file

def comprehensive_test_suite():
    """Test suite with diverse code examples"""
    
    detector = OptimizedAIGeneratedCodeDetector()
    
    test_cases = [
        {
            'name': '🤖 CASE 1: Classic AI-Generated (ChatGPT style)',
            'expected': 'HIGH AI (80-95%)',
            'code': '''
def calculate_prime_numbers(upper_limit):
    """
    Generate all prime numbers up to the specified upper limit.
    
    This function implements the Sieve of Eratosthenes algorithm
    to efficiently find all prime numbers up to upper_limit.
    
    Args:
        upper_limit (int): The upper boundary for prime number generation
        
    Returns:
        list: A list containing all prime numbers up to upper_limit
    """
    if upper_limit < 2:
        return []
    
    is_prime = [True] * (upper_limit + 1)
    is_prime[0] = is_prime[1] = False
    
    for current_number in range(2, int(upper_limit ** 0.5) + 1):
        if is_prime[current_number]:
            for multiple in range(current_number * current_number, upper_limit + 1, current_number):
                is_prime[multiple] = False
    
    prime_numbers = [number for number, is_prime_flag in enumerate(is_prime) if is_prime_flag]
    return prime_numbers
            '''
        },
        
        {
            'name': '👤 CASE 2: Human-Written (Messy, Quick Code)',
            'expected': 'LOW/CLEAN (0-30%)',
            'code': '''
def primes(n):
    # quick prime check
    res=[]
    for i in range(2,n):
        ok=True
        for j in range(2,i):
            if i%j==0:
                ok=False
                break
        if ok:res.append(i)
    return res
            '''
        },
        
        {
            'name': '🔄 CASE 3: Human Code with AI Refactoring',
            'expected': 'MEDIUM (45-65%)',
            'code': '''
def find_duplicates(array):
    """Find duplicate elements in array"""
    seen = set()
    duplicates = []
    
    for element in array:
        if element in seen:
            if element not in duplicates:
                duplicates.append(element)
        else:
            seen.add(element)
    
    return duplicates
            '''
        },
        
        {
            'name': '🧪 CASE 4: Competitive Programming Style (Human)',
            'expected': 'LOW/MEDIUM (20-40%)',
            'code': '''
def solve():
    n = int(input())
    a = list(map(int, input().split()))
    
    # dp[i] = max sum ending at i
    dp = [0]*n
    dp[0] = max(0, a[0])
    
    for i in range(1,n):
        dp[i] = max(dp[i-1]+a[i], a[i], 0)
    
    print(max(dp))

solve()
            '''
        },
        
        {
            'name': ' CASE 5: AI with Deliberate "Humanization"',
            'expected': 'MEDIUM-HIGH (55-75%)',
            'code': '''
def binary_search(arr, target):
    """Binary search implementation"""
    left, right = 0, len(arr) - 1
    
    while left <= right:
        mid = (left + right) // 2
        
        if arr[mid] == target:
            return mid
        elif arr[mid] < target:
            left = mid + 1
        else:
            right = mid - 1
    
    return -1  # not found
            '''
        },
        
        {
            'name': '💀 CASE 6: Copy-Pasted from StackOverflow',
            'expected': 'MEDIUM (40-60%)',
            'code': '''
def quicksort(arr):
    if len(arr) <= 1:
        return arr
    pivot = arr[len(arr) // 2]
    left = [x for x in arr if x < pivot]
    middle = [x for x in arr if x == pivot]
    right = [x for x in arr if x > pivot]
    return quicksort(left) + middle + quicksort(right)
            '''
        },
        
        {
            'name': '🔥 CASE 7: Expert Human (Clean but Natural)',
            'expected': 'LOW (10-35%)',
            'code': '''
def merge_intervals(intervals):
    if not intervals:
        return []
    
    intervals.sort(key=lambda x: x[0])
    merged = [intervals[0]]
    
    for current in intervals[1:]:
        last = merged[-1]
        
        if current[0] <= last[1]:
            merged[-1] = [last[0], max(last[1], current[1])]
        else:
            merged.append(current)
    
    return merged
            '''
        }
    ]
    
    print("\n" + "="*80)
    print("🧪 COMPREHENSIVE AI DETECTION TEST SUITE")
    print("="*80)
    
    results_summary = []
    
    for idx, test in enumerate(test_cases, 1):
        print(f"\n\n{'='*80}")
        print(f"TEST {idx}/7: {test['name']}")
        print(f"Expected: {test['expected']}")
        print(f"{'='*80}")
        
        result = detector.detect_ai_generated(test['code'])
        
        print(f"\n\n{' RESULTS':^80}")
        print(f"{'='*80}")
        print(f"AI Probability: {result['ai_probability']:.1%}")
        print(f"Risk Level: {result['risk_level']}")
        print(f"Verdict: {result['verdict']}")
        print(f"Is AI Generated: {result['is_ai_generated']}")
        print(f"Conflict Detected: {result.get('conflict_detected', False)}")
        
        print(f"\ Score Breakdown:")
        print(f"   Perplexity: {result['score_breakdown']['perplexity_score']:.3f}")
        print(f"   AST: {result['score_breakdown']['ast_score']:.3f}")
        print(f"   Pattern: {result['score_breakdown']['pattern_score']:.3f}")
        
        print(f"\n Detailed Metrics:")
        print(f"   Perplexity Value: {result['metrics']['perplexity']:.2f}")
        print(f"   Perplexity Level: {result['metrics']['perplexity_level']}")
        
        # Store for summary
        results_summary.append({
            'name': test['name'],
            'expected': test['expected'],
            'probability': result['ai_probability'],
            'verdict': result['risk_level'],
            'perplexity': result['metrics']['perplexity']
        })
        
        print(f"{'='*80}")
        
        # Pause between tests for readability
        import time
        time.sleep(0.5)
    
    # Print summary table
    print(f"\n\n{'='*80}")
    print(f"📋 TEST RESULTS SUMMARY")
    print(f"{'='*80}\n")
    
    print(f"{'Test':<45} {'Expected':<20} {'Actual':<15} {'Perplexity':<12}")
    print(f"{'-'*45} {'-'*20} {'-'*15} {'-'*12}")
    
    for r in results_summary:
        name_short = r['name'][:43]
        print(f"{name_short:<45} {r['expected']:<20} {r['probability']*100:>6.1f}% ({r['verdict']:<4}) {r['perplexity']:>8.2f}")
    
    print(f"\n{'='*80}")
    print(" All tests completed!")
    print(f"{'='*80}\n")


if __name__ == "__main__":
    comprehensive_test_suite()


In [ ]:
import hashlib
import difflib
from functools import lru_cache
from concurrent.futures import ThreadPoolExecutor
import numpy as np
import ast
import re
from typing import Dict, List, Optional, Tuple


class OptimizedAIGeneratedCodeDetector:
    """
    Advanced normalization for plagiarism detection
    Removes superficial differences while preserving semantic structure
    """
    
    @staticmethod
    def normalize_code(code: str, level: str = 'medium') -> str:
        """
        Multi-level normalization
        
        Args:
            code: Source code string
            level: 'light', 'medium', 'aggressive'
        """
        if level == 'light':
            return OptimizedAIGeneratedCodeDetector._light_normalize(code)
        elif level == 'medium':
            return OptimizedAIGeneratedCodeDetector._medium_normalize(code)
        else:
            return OptimizedAIGeneratedCodeDetector._aggressive_normalize(code)
    
    @staticmethod
    def _light_normalize(code: str) -> str:
        """Remove comments and extra whitespace only"""
        # Remove single-line comments
        code = re.sub(r'#.*$', '', code, flags=re.MULTILINE)
        # Remove docstrings
        code = re.sub(r'"""[\s\S]*?"""', '', code)
        code = re.sub(r"'''[\s\S]*?'''", '', code)
        # Normalize whitespace
        code = re.sub(r'\s+', ' ', code).strip()
        return code
    
    @staticmethod
    def _medium_normalize(code: str) -> str:
        """Remove comments, normalize whitespace and variable names"""
        code = OptimizedAIGeneratedCodeDetector._light_normalize(code)
        
        try:
            tree = ast.parse(code)
            
            class VariableRenamer(ast.NodeTransformer):
                def __init__(self):
                    self.var_map = {}
                    self.counter = 0
                    self.builtins = {'print', 'len', 'range', 'str', 'int', 'float', 
                                   'list', 'dict', 'set', 'sum', 'max', 'min', 'input'}
                
                def visit_Name(self, node):
                    if node.id not in self.builtins:
                        if node.id not in self.var_map:
                            self.var_map[node.id] = f'v{self.counter}'
                            self.counter += 1
                        node.id = self.var_map[node.id]
                    return node
                
                def visit_FunctionDef(self, node):
                    if node.name not in self.var_map:
                        self.var_map[node.name] = f'f{self.counter}'
                        self.counter += 1
                    node.name = self.var_map[node.name]
                    self.generic_visit(node)
                    return node
            
            renamer = VariableRenamer()
            tree = renamer.visit(tree)
            code = ast.unparse(tree)
        except:
            pass  # Return light normalized if AST fails
        
        return code
    
    @staticmethod
    def _aggressive_normalize(code: str) -> str:
        """Full normalization including structure simplification"""
        code = OptimizedAIGeneratedCodeDetector._medium_normalize(code)
        
        # Additional aggressive normalizations
        code = re.sub(r'\s+', '', code)  # Remove ALL whitespace
        code = code.lower()  # Lowercase everything
        
        return code


class EnhancedPlagiarismDetector:
    """
    Production-grade plagiarism detection with multiple strategies
    """
    
    def __init__(self, pattern_db_path: Optional[str] = None):
        """
        Initialize plagiarism detector
        
        Args:
            pattern_db_path: Path to custom pattern database (optional)
        """
        self.normalizer = OptimizedAIGeneratedCodeDetector()
        self.pattern_hashes = {}
        self.patterns_by_hash = {}
        self.pattern_database = {}
        
        # Load common patterns
        self._load_common_patterns()
        
        # Load custom patterns if provided
        if pattern_db_path:
            self._load_custom_patterns(pattern_db_path)
    
    def _load_common_patterns(self):
        """Load expanded set of common algorithm patterns"""
        
        patterns = {
            # Sorting algorithms
            'quicksort_basic': {
                'code': 'def quicksort(arr): if len(arr) <= 1: return arr pivot = arr[len(arr) // 2]',
                'category': 'sorting',
                'sources': ['stackoverflow', 'github', 'geeksforgeeks']
            },
            'bubble_sort': {
                'code': 'def bubble_sort(arr): for i in range(len(arr)): for j in range(len(arr)-i-1): if arr[j] > arr[j+1]:',
                'category': 'sorting',
                'sources': ['stackoverflow', 'w3schools']
            },
            'merge_sort': {
                'code': 'def merge_sort(arr): if len(arr) > 1: mid = len(arr) // 2 left = arr[:mid]',
                'category': 'sorting',
                'sources': ['github', 'programiz']
            },
            'insertion_sort': {
                'code': 'def insertion_sort(arr): for i in range(1, len(arr)): key = arr[i] j = i-1',
                'category': 'sorting',
                'sources': ['stackoverflow']
            },
            
            # Searching algorithms
            'binary_search': {
                'code': 'def binary_search(arr, target): left = 0 right = len(arr) - 1 while left <= right:',
                'category': 'searching',
                'sources': ['leetcode', 'github']
            },
            'linear_search': {
                'code': 'def linear_search(arr, target): for i in range(len(arr)): if arr[i] == target: return i',
                'category': 'searching',
                'sources': ['stackoverflow']
            },
            
            # Dynamic programming
            'fibonacci_recursive': {
                'code': 'def fibonacci(n): if n <= 1: return n return fibonacci(n-1) + fibonacci(n-2)',
                'category': 'dynamic_programming',
                'sources': ['common_algorithm', 'geeksforgeeks']
            },
            'fibonacci_dp': {
                'code': 'def fibonacci(n): dp = [0, 1] for i in range(2, n+1): dp.append(dp[i-1] + dp[i-2])',
                'category': 'dynamic_programming',
                'sources': ['leetcode']
            },
            'knapsack': {
                'code': 'def knapsack(weights, values, capacity): dp = [[0 for _ in range(capacity+1)] for _ in range(len(weights)+1)]',
                'category': 'dynamic_programming',
                'sources': ['geeksforgeeks', 'github']
            },
            
            # Data structures
            'linked_list_node': {
                'code': 'class Node: def __init__(self, data): self.data = data self.next = None',
                'category': 'data_structures',
                'sources': ['stackoverflow', 'github']
            },
            'binary_tree_node': {
                'code': 'class TreeNode: def __init__(self, val): self.val = val self.left = None self.right = None',
                'category': 'data_structures',
                'sources': ['leetcode', 'github']
            },
            
            # Graph algorithms
            'dfs_recursive': {
                'code': 'def dfs(graph, node, visited): if node not in visited: visited.add(node) for neighbor in graph[node]:',
                'category': 'graph',
                'sources': ['geeksforgeeks']
            },
            'bfs': {
                'code': 'def bfs(graph, start): visited = set() queue = [start] while queue: node = queue.pop(0)',
                'category': 'graph',
                'sources': ['stackoverflow', 'github']
            }
        }
        
        # Pre-compute hashes for all patterns
        for name, info in patterns.items():
            normalized = self.normalizer.normalize_code(info['code'], level='medium')
            pattern_hash = hashlib.md5(normalized.encode()).hexdigest()
            
            self.pattern_hashes[name] = pattern_hash
            self.patterns_by_hash[pattern_hash] = {
                'name': name,
                'code': info['code'],
                'normalized': normalized,
                'category': info['category'],
                'sources': info['sources']
            }
            self.pattern_database[name] = info
    
    def _load_custom_patterns(self, db_path: str):
        """Load custom patterns from file/database"""
        # Placeholder for custom pattern loading
        # In production, this would load from a database or file
        pass
    
    @lru_cache(maxsize=2000)
    def _cached_normalize(self, code: str, level: str) -> str:
        """Cached normalization for performance"""
        return self.normalizer.normalize_code(code, level)
    
    def check_pattern_database(self, code: str) -> Dict:
        """
        Check code against known pattern database
        
        Returns:
            Dict with matches and similarity scores
        """
        matches = []
        
        # Try multiple normalization levels
        for level in ['medium', 'aggressive']:
            normalized_code = self._cached_normalize(code, level)
            code_hash = hashlib.md5(normalized_code.encode()).hexdigest()
            
            # Exact hash match
            if code_hash in self.patterns_by_hash:
                pattern_info = self.patterns_by_hash[code_hash]
                matches.append({
                    'pattern_name': pattern_info['name'],
                    'category': pattern_info['category'],
                    'similarity': 1.0,
                    'sources': pattern_info['sources'],
                    'match_type': 'exact',
                    'normalization_level': level
                })
                continue
            
            # Fuzzy matching for similar patterns
            for pattern_hash, pattern_info in self.patterns_by_hash.items():
                pattern_normalized = pattern_info['normalized']
                
                # Quick length check to skip obviously different codes
                len_ratio = len(normalized_code) / max(len(pattern_normalized), 1)
                if len_ratio < 0.5 or len_ratio > 2.0:
                    continue
                
                # Calculate similarity
                similarity = difflib.SequenceMatcher(
                    None, 
                    normalized_code, 
                    pattern_normalized
                ).ratio()
                
                if similarity > 0.75:  # Threshold for similarity
                    matches.append({
                        'pattern_name': pattern_info['name'],
                        'category': pattern_info['category'],
                        'similarity': similarity,
                        'sources': pattern_info['sources'],
                        'match_type': 'similar',
                        'normalization_level': level
                    })
        
        # Remove duplicate matches
        seen = set()
        unique_matches = []
        for match in matches:
            key = (match['pattern_name'], match['match_type'])
            if key not in seen:
                seen.add(key)
                unique_matches.append(match)
        
        return unique_matches
    
    def compare_submissions(self, code1: str, code2: str) -> Dict:
        """
        Compare two code submissions for similarity
        
        Args:
            code1: First code submission
            code2: Second code submission
            
        Returns:
            Dict with similarity analysis
        """
        results = {}
        
        # Compare at multiple normalization levels
        for level in ['light', 'medium', 'aggressive']:
            norm1 = self._cached_normalize(code1, level)
            norm2 = self._cached_normalize(code2, level)
            
            # Calculate similarity
            similarity = difflib.SequenceMatcher(None, norm1, norm2).ratio()
            
            results[f'{level}_similarity'] = similarity
        
        # Structural similarity using AST
        try:
            tree1 = ast.parse(code1)
            tree2 = ast.parse(code2)
            
            # Compare AST structures
            dump1 = ast.dump(tree1)
            dump2 = ast.dump(tree2)
            
            structural_similarity = difflib.SequenceMatcher(None, dump1, dump2).ratio()
            results['structural_similarity'] = structural_similarity
        except:
            results['structural_similarity'] = 0.0
        
        # Overall assessment
        results['max_similarity'] = max(
            results.get('light_similarity', 0),
            results.get('medium_similarity', 0),
            results.get('aggressive_similarity', 0)
        )
        
        results['is_similar'] = results['max_similarity'] > 0.85
        
        return results
    
    def detect_plagiarism(self, code: str, submission_database: Optional[List[Dict]] = None) -> Dict:
        """
        Comprehensive plagiarism detection
        
        Args:
            code: Code to analyze
            submission_database: Optional list of previous submissions to compare against
                               Each dict should have {'code': str, 'user_id': str, 'timestamp': str}
        
        Returns:
            Comprehensive plagiarism analysis
        """
        results = {
            'is_plagiarized': False,
            'confidence': 0.0,
            'pattern_matches': [],
            'submission_matches': [],
            'sources_found': [],
            'risk_level': 'NONE'
        }
        
        # Check against common patterns
        pattern_matches = self.check_pattern_database(code)
        results['pattern_matches'] = pattern_matches
        
        if pattern_matches:
            max_pattern_similarity = max(m['similarity'] for m in pattern_matches)
            results['sources_found'] = list(set(
                source for match in pattern_matches for source in match['sources']
            ))
            
            # Determine if pattern match is problematic
            if max_pattern_similarity >= 0.95:
                results['is_plagiarized'] = True
                results['confidence'] = max_pattern_similarity
                results['risk_level'] = 'HIGH'
                results['reason'] = f"Exact copy of common pattern"
            elif max_pattern_similarity >= 0.85:
                results['is_plagiarized'] = True
                results['confidence'] = max_pattern_similarity
                results['risk_level'] = 'MEDIUM'
                results['reason'] = f"Very similar to common pattern"
        
        # Check against submission database if provided
        if submission_database:
            for submission in submission_database:
                comparison = self.compare_submissions(code, submission['code'])
                
                if comparison['max_similarity'] > 0.85:
                    results['submission_matches'].append({
                        'user_id': submission.get('user_id', 'unknown'),
                        'timestamp': submission.get('timestamp', 'unknown'),
                        'similarity': comparison['max_similarity'],
                        'details': comparison
                    })
                    
                    if comparison['max_similarity'] > results['confidence']:
                        results['is_plagiarized'] = True
                        results['confidence'] = comparison['max_similarity']
                        results['risk_level'] = 'HIGH' if comparison['max_similarity'] > 0.95 else 'MEDIUM'
                        results['reason'] = f"Matches another submission"
        
        # Final assessment
        if not results['is_plagiarized'] and results['pattern_matches']:
            # Some similarity but not plagiarism
            avg_similarity = np.mean([m['similarity'] for m in pattern_matches])
            if avg_similarity > 0.6:
                results['risk_level'] = 'LOW'
                results['reason'] = "Uses common algorithm patterns (acceptable)"
        
        return results


class OptimizedCodeChecker:
    """
    Unified code analysis system with AI detection and plagiarism checking
    """
    
    def __init__(self):
        print("Initializing Code Analysis System...")
        self.ai_detector = OptimizedAIGeneratedCodeDetector()
        self.plagiarism_detector = EnhancedPlagiarismDetector()
        self.executor = ThreadPoolExecutor(max_workers=2)
        print("Code Analysis System Ready\n")
    
    def analyze_code(self, 
                    code: str, 
                    submission_database: Optional[List[Dict]] = None,
                    parallel: bool = True) -> Dict:
        """
        Comprehensive code analysis
        
        Args:
            code: Code to analyze
            submission_database: Optional previous submissions for plagiarism comparison
            parallel: Run detections in parallel
        
        Returns:
            Complete analysis results
        """
        
        if parallel:
            # Run both detections in parallel
            future_ai = self.executor.submit(self.ai_detector.detect_ai_generated, code)
            future_plag = self.executor.submit(
                self.plagiarism_detector.detect_plagiarism, 
                code, 
                submission_database
            )
            
            ai_result = future_ai.result()
            plag_result = future_plag.result()
        else:
            ai_result = self.ai_detector.detect_ai_generated(code)
            plag_result = self.plagiarism_detector.detect_plagiarism(code, submission_database)
        
        # Combine results
        overall_suspicious = (
            ai_result['is_ai_generated'] or 
            plag_result['is_plagiarized']
        )
        
        # Determine overall risk level
        risk_levels = {'NONE': 0, 'LOW': 1, 'MEDIUM': 2, 'HIGH': 3, 'CRITICAL': 4}
        ai_risk = risk_levels.get(ai_result['risk_level'], 0)
        plag_risk = risk_levels.get(plag_result['risk_level'], 0)
        
        overall_risk_value = max(ai_risk, plag_risk)
        overall_risk = [k for k, v in risk_levels.items() if v == overall_risk_value][0]
        
        return {
            'overall_suspicious': overall_suspicious,
            'overall_risk_level': overall_risk,
            'ai_detection': ai_result,
            'plagiarism_detection': plag_result,
            'recommendation': self._generate_recommendation(ai_result, plag_result),
            'action': self._determine_action(ai_result, plag_result)
        }
    
    def _generate_recommendation(self, ai_result: Dict, plag_result: Dict) -> str:
        """Generate human-readable recommendation"""
        
        if ai_result['is_ai_generated'] and plag_result['is_plagiarized']:
            return (f"CRITICAL: Code shows both AI generation "
                   f"(confidence: {ai_result['ai_probability']:.1%}) "
                   f"and plagiarism (similarity: {plag_result['confidence']:.1%})")
        
        elif ai_result['is_ai_generated']:
            return (f"AI DETECTED: Code likely AI-generated "
                   f"(confidence: {ai_result['ai_probability']:.1%}). "
                   f"Recommend discussing with student about AI usage policies.")
        
        elif plag_result['is_plagiarized']:
            sources = plag_result.get('sources_found', [])
            if sources:
                source_str = ', '.join(sources)
                return f"PLAGIARISM DETECTED: Code matches known sources ({source_str})"
            elif plag_result.get('submission_matches'):
                return f"PLAGIARISM DETECTED: Code matches other student submissions"
            else:
                return "PLAGIARISM DETECTED: Code matches existing patterns"
        
        elif ai_result['ai_probability'] > 0.4:
            return "MODERATE RISK: Some AI-like patterns detected. May warrant review."
        
        elif plag_result['risk_level'] == 'LOW':
            return "ACCEPTABLE: Code uses common algorithm patterns (normal)"
        
        else:
            return "CLEAN: No significant issues detected"
    
    def _determine_action(self, ai_result: Dict, plag_result: Dict) -> str:
        """Determine recommended action"""
        
        if ai_result['is_ai_generated'] and plag_result['is_plagiarized']:
            return "BLOCK_AND_REPORT"
        elif plag_result['is_plagiarized'] and plag_result['risk_level'] == 'HIGH':
            return "INVESTIGATE"
        elif ai_result['is_ai_generated'] and ai_result['risk_level'] == 'HIGH':
            return "FLAG_FOR_REVIEW"
        elif ai_result['ai_probability'] > 0.4 or plag_result['risk_level'] != 'NONE':
            return "MONITOR"
        else:
            return "ACCEPT"
    
    def clear_cache(self):
        """Clear all LRU caches"""
        if hasattr(self.plagiarism_detector, '_cached_normalize'):
            self.plagiarism_detector._cached_normalize.cache_clear()


# Testing
if __name__ == "__main__":
    checker = OptimizedCodeChecker()
    
    test_code = '''
def bubble_sort(arr):
    for i in range(len(arr)):
        for j in range(len(arr) - i - 1):
            if arr[j] > arr[j+1]:
                arr[j], arr[j+1] = arr[j+1], arr[j]
    return arr
    '''
    
    print("="*70)
    print("COMPREHENSIVE CODE ANALYSIS TEST")
    print("="*70)
    
    result = checker.analyze_code(test_code)
    
    print(f"\nOVERALL ASSESSMENT:")
    print(f"  Suspicious: {result['overall_suspicious']}")
    print(f"  Risk Level: {result['overall_risk_level']}")
    print(f"  Action: {result['action']}")
    print(f"  Recommendation: {result['recommendation']}")
    
    print(f"\nAI DETECTION:")
    print(f"  Probability: {result['ai_detection']['ai_probability']:.1%}")
    print(f"  Risk: {result['ai_detection']['risk_level']}")
    
    print(f"\nPLAGIARISM DETECTION:")
    print(f"  Is Plagiarized: {result['plagiarism_detection']['is_plagiarized']}")
    print(f"  Confidence: {result['plagiarism_detection']['confidence']:.1%}")
    print(f"  Risk: {result['plagiarism_detection']['risk_level']}")
    if result['plagiarism_detection']['pattern_matches']:
        print(f"  Pattern Matches: {len(result['plagiarism_detection']['pattern_matches'])}")
    
    print("="*70)


Initializing Code Analysis System...


NameError: name 'OptimizedAIGeneratedCodeDetector' is not defined